In [3]:
import pandas as pd
import os

### Data Preparation

In [7]:
companies = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA']

# Create an empty DataFrame to store combined data
combined_stock_data = pd.DataFrame()

# Loop through each file, load it, and append it to the combined DataFrame
for company in companies:
    file_path = f'../Data/yfinance_data/{company}_historical_data.csv'  # File path for each company's CSV
    if os.path.exists(file_path):  # Check if the file exists
        temp_df = pd.read_csv(file_path)
        temp_df['company'] = company  # Add a 'company' column to identify which company it is
        combined_stock_data = pd.concat([combined_stock_data, temp_df], ignore_index=True)
    else:
        print(f"File not found: {file_path}")

# Check the combined DataFrame
combined_stock_data.head()


,Date,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,company
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,0.098943,469033600,0.0,0.0,AAPL
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093781,175884800,0.0,0.0,AAPL
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086898,105728000,0.0,0.0,AAPL
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089049,86441600,0.0,0.0,AAPL
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091630,73449600,0.0,0.0,AAPL


In [8]:
news_df = pd.read_csv('../Data/cleaned_financial_news.csv')

In [10]:
news_df.head()

,headline,url,publisher,date,stock,headline_length
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,39
1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,42
2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A,29
3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A,44
4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A,87


In [12]:
news_df['date'] = pd.to_datetime(news_df['date'])
combined_stock_data['Date'] = pd.to_datetime(combined_stock_data['Date'])

In [13]:
trading_days = combined_stock_data['Date'].unique()
news_df = news_df[news_df['date'].isin(trading_days)]

In [14]:
aligned_news_df = news_df.merge(combined_stock_data, on=['date', 'company'], how='inner')
aligned_news_df.head()

KeyError: 'date'